# AFL Tipping Predictor Working Notebook
- Based off work completed by David Sheehan for Soccer
- By Glen Willis

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn
from scipy.stats import poisson, skellam
import os

In [2]:
# Load training data
afl_results_2009_to_2018 = pd.read_csv(r"C:\Users\Beefsports\Documents\GitHub\AFLTippingPredictor\SourceData\AFL2009to2018.csv", encoding = "ISO-8859-1")
afl_results_2009_to_2018_results = pd.DataFrame(afl_results_2009_to_2018)

In [3]:
afl_results_2009_to_2018.head()

,Date,Kick Off (local),Home Team,Away Team,Venue,Home Score,Away Score,Play Off Game?,Home Goals,Home Behinds,...,Total Score Close,Total Score Over Open,Total Score Over Min,Total Score Over Max,Total Score Over Close,Total Score Under Open,Total Score Under Min,Total Score Under Max,Total Score Under Close,Notes
0,29-Sep-18,14:30,West Coast,Collingwood,MCG,79,74,Y,11,13,...,162.5,1.9,1.9,1.93,1.93,1.9,1.9,1.93,1.93,NaN
1,22-Sep-18,13:20,West Coast,Melbourne,Optus Stadium,121,55,Y,18,13,...,173.5,1.9,1.9,1.90,1.90,1.9,1.9,1.90,1.90,NaN
2,21-Sep-18,19:50,Richmond,Collingwood,MCG,58,97,Y,8,10,...,161.5,1.9,1.9,1.90,1.90,1.9,1.9,1.90,1.90,NaN
3,15-Sep-18,19:25,Collingwood,GWS Giants,MCG,69,59,Y,9,15,...,150.5,1.9,1.9,1.90,1.90,1.9,1.9,1.90,1.90,NaN
4,14-Sep-18,19:50,Hawthorn,Melbourne,MCG,71,104,Y,10,11,...,165.5,1.9,1.9,1.90,1.90,1.9,1.9,1.90,1.90,NaN


In [4]:
# Only take columns we need for this
afl_results_2009_to_2018_results = afl_results_2009_to_2018[['Date', 'Home Team', 'Away Team', 'Home Score', 'Away Score', 'Home Odds', 'Away Odds', 'Play Off Game?']] 
afl_results_2009_to_2018_results = afl_results_2009_to_2018_results.rename(columns={'Home Team': 'HomeTeam', 'Away Team': 'AwayTeam', 'Play Off Game?':'FinalsGame', 'Home Score': 'HomeScore', 'Away Score': 'AwayScore'})

In [5]:
# Drop all finals games from this model as footy tipping is only completed for the home and away season
afl_results_2009_to_2018_results = afl_results_2009_to_2018_results[afl_results_2009_to_2018_results.FinalsGame != 'Y']
afl_results_2009_to_2018_results = afl_results_2009_to_2018_results.drop(columns='FinalsGame')

In [6]:
# Split out Date into Day, Month, Year columns by using - as a delimiter
afl_results_2009_to_2018_results[['Day','Month', 'Year']] = afl_results_2009_to_2018_results.Date.str.split("-",expand=True)
afl_results_2009_to_2018_results = afl_results_2009_to_2018_results.drop(columns='Date')

In [7]:
# Move Year, Month, Day to the front of the dataframe
afl_results_2009_to_2018_results = afl_results_2009_to_2018_results[ ['Day'] + [ col for col in afl_results_2009_to_2018_results.columns if col != 'Day' ] ]
afl_results_2009_to_2018_results = afl_results_2009_to_2018_results[ ['Month'] + [ col for col in afl_results_2009_to_2018_results.columns if col != 'Month' ] ]
afl_results_2009_to_2018_results = afl_results_2009_to_2018_results[ ['Year'] + [ col for col in afl_results_2009_to_2018_results.columns if col != 'Year' ] ]

In [8]:
# Convert odds to percentage
afl_results_2009_to_2018_results['HomeOddsPercent'] = [1 / home_odds for home_odds in afl_results_2009_to_2018_results['Home Odds']]
afl_results_2009_to_2018_results['AwayOddsPercent'] = [1 / home_odds for home_odds in afl_results_2009_to_2018_results['Away Odds']]

In [10]:
# Rename some of the teams so they match the testing dataset
afl_results_2009_to_2018_results = afl_results_2009_to_2018_results.replace('Adelaide','Adelaide Crows')
afl_results_2009_to_2018_results = afl_results_2009_to_2018_results.replace('Brisbane','Brisbane Lions')
afl_results_2009_to_2018_results = afl_results_2009_to_2018_results.replace('Geelong','Geelong Cats')
afl_results_2009_to_2018_results = afl_results_2009_to_2018_results.replace('Gold Coast','Gold Coast Suns')
afl_results_2009_to_2018_results = afl_results_2009_to_2018_results.replace('Sydney','Sydney Swans')
afl_results_2009_to_2018_results = afl_results_2009_to_2018_results.replace('West Coast','West Coast Eagles')

In [11]:
afl_results_2009_to_2018_results.head()

,Year,Month,Day,HomeTeam,AwayTeam,HomeScore,AwayScore,Home Odds,Away Odds,HomeOddsPercent,AwayOddsPercent
9,18,Aug,26,St Kilda,North Melbourne,94,117,3.15,1.35,0.317460,0.740741
10,18,Aug,26,Melbourne,GWS Giants,102,57,1.45,2.68,0.689655,0.373134
11,18,Aug,26,Brisbane Lions,West Coast Eagles,72,98,2.20,1.66,0.454545,0.602410
12,18,Aug,25,Carlton,Adelaide Crows,61,165,5.25,1.14,0.190476,0.877193
13,18,Aug,25,Sydney Swans,Hawthorn,74,83,2.04,1.80,0.490196,0.555556


In [12]:
# Check to confirm if Home Score and Away Score are numbers as expected
afl_results_2009_to_2018_results.applymap(np.isreal)

,Year,Month,Day,HomeTeam,AwayTeam,HomeScore,AwayScore,Home Odds,Away Odds,HomeOddsPercent,AwayOddsPercent
9,False,False,False,False,False,True,True,True,True,True,True
10,False,False,False,False,False,True,True,True,True,True,True
11,False,False,False,False,False,True,True,True,True,True,True
12,False,False,False,False,False,True,True,True,True,True,True
13,False,False,False,False,False,True,True,True,True,True,True
14,False,False,False,False,False,True,True,True,True,True,True
15,False,False,False,False,False,True,True,True,True,True,True
16,False,False,False,False,False,True,True,True,True,True,True
17,False,False,False,False,False,True,True,True,True,True,True
18,False,False,False,False,False,True,True,True,True,True,True


In [13]:
# Need to coerce the Year into a numeric value for use in filtering below
afl_results_2009_to_2018_results['Year'] = pd.to_numeric(afl_results_2009_to_2018_results['Year'],errors='coerce')

# Due to the up and down nature of footy teams over the years I'll take just the last five years of results for the model - just as a guess
afl_results_2014_to_2018_results = afl_results_2009_to_2018_results[(afl_results_2009_to_2018_results['Year'] >= 14) &
                                                                    (afl_results_2009_to_2018_results['Year'] <= 18)]
afl_results_2014_to_2018_results.head()

,Year,Month,Day,HomeTeam,AwayTeam,HomeScore,AwayScore,Home Odds,Away Odds,HomeOddsPercent,AwayOddsPercent
9,18,Aug,26,St Kilda,North Melbourne,94,117,3.15,1.35,0.317460,0.740741
10,18,Aug,26,Melbourne,GWS Giants,102,57,1.45,2.68,0.689655,0.373134
11,18,Aug,26,Brisbane Lions,West Coast Eagles,72,98,2.20,1.66,0.454545,0.602410
12,18,Aug,25,Carlton,Adelaide Crows,61,165,5.25,1.14,0.190476,0.877193
13,18,Aug,25,Sydney Swans,Hawthorn,74,83,2.04,1.80,0.490196,0.555556


In [14]:
# Importing the tools required for the Poisson regression model - average points scored
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Below if home team then make HomeTeam team and AwayTeam opponent and vice versa
afl_tipping_model_data = pd.concat([afl_results_2014_to_2018_results[['HomeTeam','AwayTeam','HomeScore']].assign(home=1).rename(
            columns={'HomeTeam':'team','AwayTeam':'opponent','HomeScore':'score'}),
           afl_results_2014_to_2018_results[['AwayTeam','HomeTeam','AwayScore']].assign(home=0).rename(
            columns={'AwayTeam':'team', 'HomeTeam':'opponent','AwayScore':'score'})])

afl_tipping_model_data.head()

,team,opponent,score,home
9,St Kilda,North Melbourne,94,1
10,Melbourne,GWS Giants,102,1
11,Brisbane Lions,West Coast Eagles,72,1
12,Carlton,Adelaide Crows,61,1
13,Sydney Swans,Hawthorn,74,1


In [15]:
# Coerce score to numeric
afl_tipping_model_data['score'] = pd.to_numeric(afl_tipping_model_data['score'],errors='coerce')
afl_tipping_model_data.head()

,team,opponent,score,home
9,St Kilda,North Melbourne,94,1
10,Melbourne,GWS Giants,102,1
11,Brisbane Lions,West Coast Eagles,72,1
12,Carlton,Adelaide Crows,61,1
13,Sydney Swans,Hawthorn,74,1


In [16]:
# Creates a poisson model using the statsmodels generalised linear model (glm) with score as the dependent variable and
# home (i.e. if home team or not), team and opponent as input variables (R-style syntax), data is above 
# and family specifies as Poisson
# .fit() fits a generalised linear model for a given family

afl_2014_to_2018_poisson_model = smf.glm(formula="score ~ home + team + opponent", data=afl_tipping_model_data, 
                        family=sm.families.Poisson()).fit()
# Gives a glm regression result summary
afl_2014_to_2018_poisson_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                  score   No. Observations:                 1978
Model:                            GLM   Df Residuals:                     1942
Model Family:                 Poisson   Df Model:                           35
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -12712.
Date:                Thu, 26 Mar 2020   Deviance:                       13053.
Time:                        07:16:46   Pearson chi2:                 1.29e+04
No. Iterations:                     4   Covariance Type:             nonrobust
=================================================================================================
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                         4.5603      0.015    313.126      0.000       4.532       4.589
team[T.Brisbane Lions]           -0.2642      0.014    -18.324      0.000      -0.292      -0.236
team[T.Carlton]                  -0.3466      0.015    -23.522      0.000      -0.375      -0.318
team[T.Collingwood]              -0.1556      0.014    -11.147      0.000      -0.183      -0.128
team[T.Essendon]                 -0.2309      0.014    -16.193      0.000      -0.259      -0.203
team[T.Fremantle]                -0.2618      0.014    -18.210      0.000      -0.290      -0.234
team[T.GWS Giants]               -0.1109      0.014     -8.022      0.000      -0.138      -0.084
team[T.Geelong Cats]             -0.0832      0.014     -6.063      0.000      -0.110      -0.056
team[T.Gold Coast Suns]          -0.2942      0.014    -20.296      0.000      -0.323      -0.266
team[T.Hawthorn]                 -0.0305      0.014     -2.257      0.024      -0.057      -0.004
team[T.Melbourne]                -0.2050      0.014    -14.495      0.000      -0.233      -0.177
team[T.North Melbourne]          -0.1251      0.014     -9.036      0.000      -0.152      -0.098
team[T.Port Adelaide]            -0.1017      0.014     -7.377      0.000      -0.129      -0.075
team[T.Richmond]                 -0.1604      0.014    -11.500      0.000      -0.188      -0.133
team[T.St Kilda]                 -0.2537      0.014    -17.652      0.000      -0.282      -0.226
team[T.Sydney Swans]             -0.0945      0.014     -6.894      0.000      -0.121      -0.068
team[T.West Coast Eagles]        -0.0726      0.014     -5.323      0.000      -0.099      -0.046
team[T.Western Bulldogs]         -0.2072      0.014    -14.666      0.000      -0.235      -0.179
opponent[T.Brisbane Lions]        0.2507      0.014     18.016      0.000       0.223       0.278
opponent[T.Carlton]               0.1415      0.014      9.924      0.000       0.114       0.169
opponent[T.Collingwood]           0.0142      0.015      0.962      0.336      -0.015       0.043
opponent[T.Essendon]              0.0786      0.014      5.427      0.000       0.050       0.107
opponent[T.Fremantle]             0.0044      0.015      0.297      0.766      -0.024       0.033
opponent[T.GWS Giants]            0.0059      0.015      0.403      0.687      -0.023       0.035
opponent[T.Geelong Cats]         -0.0752      0.015     -4.983      0.000      -0.105      -0.046
opponent[T.Gold Coast Suns]       0.1683      0.014     11.882      0.000       0.141       0.196
opponent[T.Hawthorn]             -0.0518      0.015     -3.460      0.001      -0.081      -0.022
opponent[T.Melbourne]             0.0372      0.015      2.542      0.011       0.009       0.066
opponent[T.North Melbourne]       0.0365      0.015      2.488      0.013       0.008       0.065
opponent[T.Port Adelaide] 

In [17]:
# Make a function to simulate any match

def simulate_match(tipping_model, homeTeam, awayTeam, max_score=250):
    home_score_avg = tipping_model.predict(pd.DataFrame(data={'team': homeTeam,
                                                             'opponent': awayTeam,'home':1},
                                                       index=[1])).values[0]
    away_score_avg = tipping_model.predict(pd.DataFrame(data={'team': awayTeam,
                                                             'opponent': homeTeam,
                                                             'home':0},
                                                       index=[1])).values[0]
    # List comprehension to calculate the predicted team matrix (Rows are home team, Columns are away team and diagonal is chance of a draw)
    team_pred = [[poisson.pmf(i, team_avg) for i in range (0, max_score+1)] for team_avg in [home_score_avg, away_score_avg]]
    return(np.outer(np.array(team_pred[0]), np.array(team_pred[1])))

In [21]:
# Load testing data
afl_results_2019 = pd.read_csv(r"C:\Users\Beefsports\Documents\GitHub\AFLTippingPredictor\SourceData\afl-2019-AUSEasternStandardTime_results.csv", encoding = "ISO-8859-1")
afl_results_2019_results = pd.DataFrame(afl_results_2019)
afl_results_2019_results.head()

,Round Number,Date,Location,Home Team,Away Team,Result
0,1,21/03/2019 19:25,MCG,Carlton,Richmond,64 - 97
1,1,22/03/2019 19:50,MCG,Collingwood,Geelong Cats,65 - 72
2,1,23/03/2019 13:45,MCG,Melbourne,Port Adelaide,61 - 87
3,1,23/03/2019 16:35,Adelaide Oval,Adelaide Crows,Hawthorn,55 - 87
4,1,23/03/2019 19:25,Marvel Stadium,Western Bulldogs,Sydney Swans,82 - 65


In [22]:
afl_results_2019_scores = afl_results_2019[['Home Team', 'Away Team', 'Result']] # Only take Home Team, Away Team and Result
afl_results_2019_scores = afl_results_2019_scores.rename(columns={'Home Team': 'HomeTeam', 'Away Team': 'AwayTeam', 'Result':'FullScore'})
afl_results_2019_scores.head()

,HomeTeam,AwayTeam,FullScore
0,Carlton,Richmond,64 - 97
1,Collingwood,Geelong Cats,65 - 72
2,Melbourne,Port Adelaide,61 - 87
3,Adelaide Crows,Hawthorn,55 - 87
4,Western Bulldogs,Sydney Swans,82 - 65


In [23]:
afl_results_2019_split_scores = pd.DataFrame(afl_results_2019_scores) # Make pandas dataframe
# Split out FullScore into Home Score and AwayScore columns by using - as a delimiter
afl_results_2019_split_scores[['HomeScore','AwayScore']] = afl_results_2019_split_scores.FullScore.str.split(" - ",expand=True)
afl_results_2019_split_scores.head()

,HomeTeam,AwayTeam,FullScore,HomeScore,AwayScore
0,Carlton,Richmond,64 - 97,64,97
1,Collingwood,Geelong Cats,65 - 72,65,72
2,Melbourne,Port Adelaide,61 - 87,61,87
3,Adelaide Crows,Hawthorn,55 - 87,55,87
4,Western Bulldogs,Sydney Swans,82 - 65,82,65


In [24]:
# Drop FullScore column because we don't need it anymore
afl_results_2019_split_scores = afl_results_2019_split_scores.drop(columns='FullScore')
afl_results_2019_split_scores.head()

,HomeTeam,AwayTeam,HomeScore,AwayScore
0,Carlton,Richmond,64,97
1,Collingwood,Geelong Cats,65,72
2,Melbourne,Port Adelaide,61,87
3,Adelaide Crows,Hawthorn,55,87
4,Western Bulldogs,Sydney Swans,82,65


In [25]:
# Need to coerce the HomeScore and AwayScore results to numeric values as previously were strings - won't work with the poisson model
afl_results_2019_split_scores['HomeScore'] = pd.to_numeric(afl_results_2019_split_scores['HomeScore'],errors='coerce')
afl_results_2019_split_scores['AwayScore'] = pd.to_numeric(afl_results_2019_split_scores['AwayScore'],errors='coerce')

# afl_results_2019_split_scores.applymap(np.isreal)

- Values in the coef column above - analogous to the slopes in linear regression
- Take exponent of parameter values P(x) = e^coeff
- Positive value implies higher score while values closer to zero represent more neutral effects
- Home has a coef of 0.0523 - captures the fact that home teams generally score more points than the away team - e.g. e^0.0523 = 1.05 times more likely
- West Coast Eagles has coef of 0.0899 and St Kilda has -0.0803 which means the Eagles are better and St Kilda are much worse scorers
- Opponent coef values (e.g. opponent[T.St Kilda]]) penalise/reward teams based on the quality of the opposition
- Reflects defensive strength of each team
- In other words you're less likely to score against Hawthorn (with a negative value of -0.1179)
- Against Gold Coast Suns the value is 0.2222 - more likely to score

In [26]:
# Set the base assumption that the home team wins to compare against
afl_results_2019_split_scores['PredictedWinnerHomeGroundAdv'] = 'Home'

In [27]:
afl_results_2019_split_scores['HomeOddsPercentPred'] = [np.sum(np.tril(simulate_match(afl_2014_to_2018_poisson_model, homeTeam, awayTeam), -1)) for (homeTeam, awayTeam) 
                                                           in zip(afl_results_2019_split_scores['HomeTeam'], afl_results_2019_split_scores['AwayTeam'])]

afl_results_2019_split_scores['AwayOddsPercentPred'] = [np.sum(np.triu(simulate_match(afl_2014_to_2018_poisson_model, homeTeam, awayTeam), 1)) for (homeTeam, awayTeam) 
                                                           in zip(afl_results_2009_to_2018_results['HomeTeam'], afl_results_2019_split_scores['AwayTeam'])]
afl_results_2019_split_scores.head()

,HomeTeam,AwayTeam,HomeScore,AwayScore,PredictedWinnerHomeGroundAdv,HomeOddsPercentPred,AwayOddsPercentPred
0,Carlton,Richmond,64,97,Home,0.023721,0.909179
1,Collingwood,Geelong Cats,65,72,Home,0.276517,0.837307
2,Melbourne,Port Adelaide,61,87,Home,0.210859,0.995406
3,Adelaide Crows,Hawthorn,55,87,Home,0.635899,0.997477
4,Western Bulldogs,Sydney Swans,82,65,Home,0.063543,0.303484


In [28]:
afl_results_2019_split_scores['PredictedWinnerModel'] = ['Home' if homeoddspred >= awayoddspred
                                                    else 'Away' for (homeoddspred, awayoddspred) in zip(afl_results_2019_split_scores['HomeOddsPercentPred'], afl_results_2019_split_scores['AwayOddsPercentPred'])]
afl_results_2019_split_scores.head()

,HomeTeam,AwayTeam,HomeScore,AwayScore,PredictedWinnerHomeGroundAdv,HomeOddsPercentPred,AwayOddsPercentPred,PredictedWinnerModel
0,Carlton,Richmond,64,97,Home,0.023721,0.909179,Away
1,Collingwood,Geelong Cats,65,72,Home,0.276517,0.837307,Away
2,Melbourne,Port Adelaide,61,87,Home,0.210859,0.995406,Away
3,Adelaide Crows,Hawthorn,55,87,Home,0.635899,0.997477,Away
4,Western Bulldogs,Sydney Swans,82,65,Home,0.063543,0.303484,Away


In [34]:
# Add actual winner to the test data column using a list comprehension

afl_results_2019_split_scores['ActualWinner'] = ['Home' if homescore > awayscore
                                                    else 'Away' if homescore < awayscore
                                                    else 'Draw' for (homescore, awayscore)
                                                    in zip(afl_results_2019_split_scores['HomeScore'],
                                                           afl_results_2019_split_scores['AwayScore'])]

In [35]:
afl_results_2019_split_scores['PredictionCorrect'] = ['Yes' if PredictedWinner == ActualWinner
                                                    else 'No' for (PredictedWinner, ActualWinner) in zip(afl_results_2019_split_scores['PredictedWinnerModel'],
                                                                                                         afl_results_2019_split_scores['ActualWinner'])]
afl_results_2019_split_scores['PredictionCorrectHomeGround'] = ['Yes' if PredictedWinnerHomeGroundAdv == ActualWinner
                                                    else 'No' for (PredictedWinnerHomeGroundAdv, ActualWinner) in zip(afl_results_2019_split_scores['PredictedWinnerHomeGroundAdv'],
                                                                                                                      afl_results_2019_split_scores['ActualWinner'])]
afl_results_2019_split_scores.head()

,HomeTeam,AwayTeam,HomeScore,AwayScore,PredictedWinnerHomeGroundAdv,HomeOddsPercentPred,AwayOddsPercentPred,PredictedWinnerModel,ActualWinner,PredictionCorrect,PredictionCorrectHomeGround
0,Carlton,Richmond,64,97,Home,0.023721,0.909179,Away,Away,Yes,No
1,Collingwood,Geelong Cats,65,72,Home,0.276517,0.837307,Away,Away,Yes,No
2,Melbourne,Port Adelaide,61,87,Home,0.210859,0.995406,Away,Away,Yes,No
3,Adelaide Crows,Hawthorn,55,87,Home,0.635899,0.997477,Away,Away,Yes,No
4,Western Bulldogs,Sydney Swans,82,65,Home,0.063543,0.303484,Away,Home,No,Yes


In [40]:
afl_results_2019_split_scores.to_excel("afl_results_2019_split_scores.xlsx", sheet_name='2019_results_predicted')

In [37]:
afl_results_results_2019_pred_correct_map = afl_results_2019_split_scores.apply(lambda x: True if x['PredictionCorrect'] == 'Yes' else False, axis = 1)
afl_results_results_2019_pred_correct = len(afl_results_results_2019_pred_correct_map[afl_results_results_2019_pred_correct_map == True].index)
print(afl_results_results_2019_pred_correct)

118


In [38]:
afl_results_results_2019_pred_homegroud_correct_map = afl_results_2019_split_scores.apply(lambda x: True if x['PredictionCorrectHomeGround'] == 'Yes' else False, axis = 1)
afl_results_results_2019_pred_homegroud_correct = len(afl_results_results_2019_pred_homegroud_correct_map[afl_results_results_2019_pred_correct_map == True].index)
print(afl_results_results_2019_pred_homegroud_correct)

118


In [31]:
afl_results_2019_predicted_pivot_pred = afl_results_2019_split_scores.pivot_table(afl_results_2019_split_scores, columns = ['PredictionCorrect'],
                                                                                  aggfunc='size')
afl_results_2019_predicted_pivot_home = afl_results_2019_split_scores.pivot_table(afl_results_2019_split_scores, columns = ['PredictionCorrectHomeGround'],
                                                                                  aggfunc='size')

afl_results_2019_predicted_pivot_pred.loc["Total"] = afl_results_2019_predicted_pivot_pred.sum()
afl_results_2019_predicted_pivot_home.loc["Total"] = afl_results_2019_predicted_pivot_home.sum()

In [46]:
afl_results_2019_predicted_pivot_pred['PercentageCorrectPoisson'] = afl_results_2019_predicted_pivot_pred['Yes'] / afl_results_2019_predicted_pivot_pred['Total']
afl_results_2019_predicted_pivot_pred.head()

PredictionCorrect
No                           89.000000
Yes                         118.000000
Total                       207.000000
PercentageCorrectPoisson      0.570048
dtype: float64

In [47]:
afl_results_2019_predicted_pivot_home['PercentageCorrectPoisson'] = afl_results_2019_predicted_pivot_home['Yes'] / afl_results_2019_predicted_pivot_home['Total']
afl_results_2019_predicted_pivot_home.head()

PredictionCorrectHomeGround
No                           89.000000
Yes                         118.000000
Total                       207.000000
PercentageCorrectPoisson      0.570048
dtype: float64

In [48]:
combined_prediction_results_table = pd.concat([afl_results_2019_predicted_pivot_pred, afl_results_2019_predicted_pivot_home], axis = 1)
combined_prediction_results_table.columns=['PoissonPrediction','HomeGroundPrediction']
combined_prediction_results_table.head()

,PoissonPrediction,HomeGroundPrediction
No,89.000000,89.000000
Yes,118.000000,118.000000
Total,207.000000,207.000000
PercentageCorrectPoisson,0.570048,0.570048


### Discussion of results
- Based on the above the model does not outperform just picking the home team which indicates that it has a major impact on it
- Interesting that it matches the numbers Poisson prediction and HomeGroundPrediction exactly but there is not an overlap in every case i.e. will predict Away when Home predicted